In [1]:
import numpy as np
import pandas as pd
import netCDF4 as nc

## 加载nc文件和读取数据


In [2]:
nc_data = nc.Dataset("cru_ts4.05.1901.2020.tmp.dat.nc")

In [3]:
for temp_value in nc_data.variables.values():
    print("*" * 70)
    print(temp_value)

**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    long_name: longitude
    units: degrees_east
unlimited dimensions: 
current shape = (720,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    long_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (360,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 time(time)
    long_name: time
    units: days since 1900-1-1
    calendar: gregorian
unlimited dimensions: time
current shape = (1440,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 tmp(time, la

<ipython-input-3-ab2ee6009efa>:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(temp_value)
